In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=5, micro=3, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.2
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [3]:
# https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
input_filepath = "./shakespeare.txt"
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# 1. generate vocab
# 2. build mapping char->id
# 3. data -> id_data
# 4. abcd -> bcd<eos>

vocab = sorted(set(text))
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
char2idx = {char:idx for idx, char in enumerate(vocab)}
print(char2idx)

{'x': 62, 'Z': 38, 'F': 18, 'T': 32, 'n': 52, 'A': 13, '3': 9, 'b': 40, 'C': 15, 'e': 43, 'q': 55, 'K': 23, 'z': 64, 'E': 17, "'": 5, 'Y': 37, 'd': 42, 'W': 35, 'S': 31, 'p': 54, 'I': 21, 'J': 22, 'y': 63, 'X': 36, 'G': 19, '$': 3, 'N': 26, 'w': 61, 'i': 47, ' ': 1, 'h': 46, 'j': 48, 'o': 53, ',': 6, '-': 7, 'H': 20, 's': 57, 'U': 33, '!': 2, 'g': 45, 'v': 60, ':': 10, 'r': 56, 'u': 59, 'V': 34, 'O': 27, 'P': 28, ';': 11, 'L': 24, '.': 8, 'B': 14, '\n': 0, 'k': 49, 'Q': 29, 'c': 41, 't': 58, 'M': 25, 'R': 30, 'f': 44, 'm': 51, 'l': 50, '&': 4, '?': 12, 'a': 39, 'D': 16}


In [6]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [7]:
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])

[18 47 56 57 58  1 15 47 58 47]
First Citi


In [8]:
def split_input_target(id_text):
    """
    abcde -> abcd, bcde
    """
    return id_text[0:-1], id_text[1:]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1,
                                 drop_remainder = True)
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])

for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int64) F
tf.Tensor(47, shape=(), dtype=int64) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [9]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [10]:
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)


In [11]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        keras.layers.SimpleRNN(units = rnn_units,
                               stateful = True,
                               recurrent_initializer = 'glorot_uniform',
                               return_sequences = True),
        keras.layers.Dense(vocab_size),
    ])
    return model

model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
simple_rnn (SimpleRNN)       (64, None, 1024)          1311744   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [12]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [13]:
# random sampling.
# greedy, random.
sample_indices = tf.random.categorical(
    logits = example_batch_predictions[0], num_samples = 1)
print(sample_indices)
# (100, 65) -> (100, 1)
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[[18]
 [22]
 [22]
 [41]
 [24]
 [48]
 [42]
 [44]
 [37]
 [45]
 [ 6]
 [48]
 [ 6]
 [26]
 [36]
 [61]
 [ 8]
 [22]
 [63]
 [41]
 [20]
 [14]
 [20]
 [35]
 [57]
 [ 4]
 [61]
 [10]
 [10]
 [39]
 [34]
 [26]
 [43]
 [52]
 [29]
 [29]
 [43]
 [48]
 [38]
 [11]
 [13]
 [ 1]
 [64]
 [64]
 [45]
 [52]
 [45]
 [21]
 [ 5]
 [15]
 [52]
 [35]
 [16]
 [34]
 [ 4]
 [24]
 [48]
 [ 4]
 [27]
 [19]
 [39]
 [27]
 [30]
 [43]
 [16]
 [60]
 [42]
 [29]
 [42]
 [43]
 [ 1]
 [60]
 [ 9]
 [11]
 [37]
 [37]
 [36]
 [51]
 [16]
 [28]
 [60]
 [47]
 [ 8]
 [57]
 [27]
 [14]
 [16]
 [17]
 [47]
 [56]
 [18]
 [21]
 [52]
 [34]
 [22]
 [19]
 [13]
 [60]
 [21]
 [47]], shape=(100, 1), dtype=int64)
tf.Tensor(
[18 22 22 41 24 48 42 44 37 45  6 48  6 26 36 61  8 22 63 41 20 14 20 35
 57  4 61 10 10 39 34 26 43 52 29 29 43 48 38 11 13  1 64 64 45 52 45 21
  5 15 52 35 16 34  4 24 48  4 27 19 39 27 30 43 16 60 42 29 42 43  1 60
  9 11 37 37 36 51 16 28 60 47  8 57 27 14 16 17 47 56 18 21 52 34 22 19
 13 60 21 47], shape=(100,), dtype=int64)


In [14]:
print("Input: ", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Output: ", repr("".join(idx2char[target_example_batch[0]])))
print()
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  "'ertake his bad intent,\nAnd must be buried but as an intent\nThat perish'd by the way: thoughts are n"

Output:  "ertake his bad intent,\nAnd must be buried but as an intent\nThat perish'd by the way: thoughts are no"

Predictions:  "FJJcLjdfYg,j,NXw.JycHBHWs&w::aVNenQQejZ;A zzgngI'CnWDV&Lj&OGaOReDvdQde v3;YYXmDPvi.sOBDEirFInVJGAvIi"


In [15]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

model.compile(optimizer = 'adam', loss = loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1913624


In [16]:
output_dir = "./text_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

epochs = 100
history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])


Epoch 1/100
172/172 [==============================] - 44s 258ms/step - loss: 2.6715
Epoch 2/100
172/172 [==============================] - 42s 247ms/step - loss: 2.0199
Epoch 3/100
172/172 [==============================] - 44s 253ms/step - loss: 1.8145
Epoch 4/100
172/172 [==============================] - 43s 249ms/step - loss: 1.6819
Epoch 5/100
172/172 [==============================] - 43s 252ms/step - loss: 1.5952
Epoch 6/100
172/172 [==============================] - 42s 243ms/step - loss: 1.5344
Epoch 7/100
172/172 [==============================] - 42s 242ms/step - loss: 1.4893
Epoch 8/100
172/172 [==============================] - 42s 244ms/step - loss: 1.4533
Epoch 9/100
172/172 [==============================] - 42s 242ms/step - loss: 1.4256
Epoch 10/100
172/172 [==============================] - 44s 255ms/step - loss: 1.3982
Epoch 11/100
172/172 [==============================] - 44s 255ms/step - loss: 1.3769
Epoch 12/100
172/172 [==============================] - 43s 250

In [17]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_checkpoints/ckpt_100'

In [18]:
model2 = build_model(vocab_size,
                     embedding_dim,
                     rnn_units,
                     batch_size = 1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))
# start ch sequence A,
# A -> model -> b
# A.append(b) -> B
# B(Ab) -> model -> c
# B.append(c) -> C
# C(Abc) -> model -> ...
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (1, None, 1024)           1311744   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [19]:
def generate_text(model, start_string, num_generate = 1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    
    for _ in range(num_generate):
        # 1. model inference -> predictions
        # 2. sample -> ch -> text_generated.
        # 3. update input_eval
        
        # predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # predictions : [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predicted_ids: [input_eval_len, 1]
        # a b c -> b c d
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # s, x -> rnn -> s', y
        input_eval = tf.expand_dims([predicted_id], 0)
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "All: ")
print(new_text)


All: his face beauty's slanders. Well, and some unto the storm'd fear; then dreams.

CORIOLANUS:
I will be made;
Rike unwilling I will purposed-head:
And I will stir the man
And all the from your parciur flies by himself. Well;
I am the widows fair? s, Auch
An utterance on many lines, I'ld s, and thou never ed.
Some happy stride, and supll you want fairest twenty soled no geedle moise his own alone! I am toUS:
Hair undood oppasies? Or Your issues sir,
That lies alenty; and therefore from your wealth.

ROMEO:
As I beleavent an England.

TYBALT:
My love: be of thee the flesh long-prounds. Grace now, gentle more than Jove, my gracious souls,
Brief, his father,
As, by some instery assurakery live,
And finderape.

CARESLENT:
My lord of France?
Why, what I had sees to Rome, name myself; it wass'd, being all eet and being sudd it yet?

RAMILANUS:
Now, brother Clarence.
Of all run arms
By Bianciau-house your grandsistail.

PETRUCHIO:
Come, content ines, it is here?
This day, my lord, will you 